In [ ]:
from typing import *
 
import os
import json
 
from openai import OpenAI
from openai.types.chat.chat_completion import Choice
 
client = OpenAI(
    base_url="https://api.moonshot.cn/v1",
    api_key="Sorry but we cannot show this",
)
 
 
# search 工具的具体实现，这里我们只需要返回参数即可
def search_impl(arguments: Dict[str, Any]) -> Any:
    """
    在使用 Moonshot AI 提供的 search 工具的场合，只需要原封不动返回 arguments 即可，
    不需要额外的处理逻辑。
 
    但如果你想使用其他模型，并保留联网搜索的功能，那你只需要修改这里的实现（例如调用搜索
    和获取网页内容等），函数签名不变，依然是 work 的。
 
    这最大程度保证了兼容性，允许你在不同的模型间切换，并且不需要对代码有破坏性的修改。
    """
    return arguments
 
 
def chat(messages) -> Choice:
    completion = client.chat.completions.create(
        model="moonshot-v1-8k",
        messages=messages,
        temperature=0.3,
        tools=[
            {
                "type": "builtin_function",  # <-- 使用 builtin_function 声明 $web_search 函数，请在每次请求都完整地带上 tools 声明
                "function": {
                    "name": "$web_search",
                },
            }
        ]
    )
    return completion.choices[0]
 
 
def single_search(university_name):
    messages = [
        {"role": "system", "content": "你是月之暗面（Kimi）的智能客服，你负责回答用户提出的各种问题。你的回答需要返回一个JSON列表，里面包含了至少一个4个属性（support, type, name, email）的JSON对象。"},
    ]
 
    # 初始提问
    messages.append({
        "role": "user",
        "content": f"问{university_name}是否有支持多邻国英语测试的留学项目？如果有，首先返回所有这些留学项目的信息，作为JSON列表中的每一个对象；对于每一个留学项目，support返回Y，同时根据类型设置type的值（本科生项目：UG，研究生项目：PG，暑研或者交换学期：EX），否则返回support:N，且type为NI；将找到的项目名称返回为name（name统一返回英文），并且尽可能返回其项目的联系邮箱为email，联系邮箱请认真甄别，邮箱是非常重要的信息！"
    })
 
    finish_reason = None
    while finish_reason is None or finish_reason == "tool_calls":
        choice = chat(messages)
        finish_reason = choice.finish_reason
        print(finish_reason, choice)
        if finish_reason == "tool_calls":  # <-- 判断当前返回内容是否包含 tool_calls
            messages.append(choice.message)  # <-- 我们将 Kimi 大模型返回给我们的 assistant 消息也添加到上下文中，以便于下次请求时 Kimi 大模型能理解我们的诉求
            for tool_call in choice.message.tool_calls:  # <-- tool_calls 可能是多个，因此我们使用循环逐个执行
                tool_call_name = tool_call.function.name
                tool_call_arguments = json.loads(tool_call.function.arguments)  # <-- arguments 是序列化后的 JSON Object，我们需要使用 json.loads 反序列化一下
                if tool_call_name == "$web_search":
                    tool_result = search_impl(tool_call_arguments)
                else:
                    tool_result = f"Error: unable to find tool by name '{tool_call_name}'"
 
                # 使用函数执行结果构造一个 role=tool 的 message，以此来向模型展示工具调用的结果；
                # 注意，我们需要在 message 中提供 tool_call_id 和 name 字段，以便 Kimi 大模型
                # 能正确匹配到对应的 tool_call。
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": tool_call_name,
                    "content": json.dumps(tool_result),  # <-- 我们约定使用字符串格式向 Kimi 大模型提交工具调用结果，因此在这里使用 json.dumps 将执行结果序列化成字符串
                })
        print(choice.message.tool_calls)
 
    return choice.message.content  # <-- 在这里，我们才将模型生成的回复返回给用户

In [111]:
import pandas as pd
import os

csv_file = "the_final_list.csv"
universities_df = pd.read_csv(csv_file)
    
 # 打印前 5 行
print("\n文件内容的前5行:")
print(universities_df)


文件内容的前5行:
                                                   name  \
0                   Engineering Institute of Technology   
1                        Atharva College of Engineering   
2                                    West Herts College   
3             Bhagwan Parshuram Institute of Technology   
4                   Royal Holloway University of London   
...                                                 ...   
3285        City St George's, University of London (II)   
3286  City St George's, University of London (School...   
3287                            Trinity College Bristol   
3288                            Trinity College Bristol   
3289                     California College of the Arts   

                                             domains  \
0                             ['student.eit.edu.au']   
1                               ['atharvacoe.ac.in']   
2     ['student.westherts.ac.uk', 'westherts.ac.uk']   
3               ['bpitindia.edu.in', 'bpitindia.in']   


In [112]:
import json
import time

# 创建一个新的 DataFrame 用于存储结果
new_df = pd.DataFrame()

# 遍历每一行，提取 'name' 字段
for index, row in universities_df.iterrows():
    if index == 2:
        break
    time.sleep(60)
    name = row['name']
    result = single_search(name)  # 调用函数
    print(result, type(result))
    print(json.loads(result[8:-3]))
    # 将返回的 JSON 列表转换为 DataFrame 并拼接到 new_df
    result_df = pd.DataFrame(json.loads(result[8:-3]))
    new_df = pd.concat([new_df, result_df], ignore_index=True)

# 打印新 DataFrame 的前 5 行
print("\n新 DataFrame 的前5行:")
print(new_df.head())
 
# 将新 DataFrame 保存为新的 CSV 文件
new_csv_file = "processed_results.csv"
new_df.to_csv(new_csv_file, index=False)
print(f"\n新 CSV 文件已生成: {new_csv_file}")

tool_calls Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='t-web_search-67ee83565106', function=Function(arguments='{"search_result":{"search_id":"2e46c4ab67ee83561545630001fd8706"},"usage":{"total_tokens":6965}}', name='$web_search'), type='builtin_function')]))
tool_calls Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='t-web_search-67ee83581f55', function=Function(arguments='{"search_result":{"search_id":"2504a0d567ee83582b42f60001e8312f"},"usage":{"total_tokens":6965}}', name='$web_search'), type='builtin_function')]))
tool_calls Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=''

RateLimitError: Error code: 429 - {'error': {'message': 'Your account org-1e3a0d58c26f40d7afa4909fbeeb8229<ak-ezq8bd99ui1111ae71ki> request reached organization max RPM: 3, please try again after 1 seconds', 'type': 'rate_limit_reached_error'}}